In [1]:
import numpy as np
import pandas as pd
import redis
import os

In [2]:
# Connect to Redis
hostname=os.getenv('REDIS_HOST')
port=os.getenv('REDIS_PORT')
password=os.getenv('REDIS_PASSWORD')

r = redis.Redis(host=hostname, port=port, password=password)

In [3]:
r.ping()

True

In [4]:
# load the data (nzip file)
np_file = np.load('datasets/face_recognition.npz', allow_pickle=True)
np_file.files

['xvalues', 'col_names']

In [5]:
x_values = np_file['xvalues']
col_names = np_file['col_names']

df = pd.DataFrame(x_values, columns=col_names)
df

,role,name,facial_features
0,Student,Angelina Jolie,"[0.19881932, 1.4748837, -0.0979588, -0.8805054..."
1,Student,Chris Evans,"[0.35691047, -0.41491055, 0.039218444, -0.1543..."
2,Student,Scarlett Johansson,"[-0.13535985, -0.2821385, -0.42700845, -0.2825..."
3,Teacher,Barack Obama,"[-0.22177827, 1.5762169, -0.5030781, 0.1360128..."
4,Teacher,Morgan Freeman,"[0.76278937, 0.890615, 0.46873552, 0.25898102,..."


In [6]:
df['role_name'] = df['role']+'@'+df['name']
df

,role,name,facial_features,role_name
0,Student,Angelina Jolie,"[0.19881932, 1.4748837, -0.0979588, -0.8805054...",Student@Angelina Jolie
1,Student,Chris Evans,"[0.35691047, -0.41491055, 0.039218444, -0.1543...",Student@Chris Evans
2,Student,Scarlett Johansson,"[-0.13535985, -0.2821385, -0.42700845, -0.2825...",Student@Scarlett Johansson
3,Teacher,Barack Obama,"[-0.22177827, 1.5762169, -0.5030781, 0.1360128...",Teacher@Barack Obama
4,Teacher,Morgan Freeman,"[0.76278937, 0.890615, 0.46873552, 0.25898102,...",Teacher@Morgan Freeman


In [7]:
records = df[['role_name', 'facial_features']].to_dict('records')

In [8]:
len(records)

5

### Save data in Redis

In [9]:
for record in records:
    role_name = record['role_name']
    vector = record['facial_features']

    # convert numpy array to bytes
    vector_bytes = vector.tobytes()

    # save to redis
    r.hset(name='academy:register', key=role_name, value=vector_bytes)

### Retrive Records from Redis

In [10]:
retrieve_dict = r.hgetall('academy:register')
retrieve_series = pd.Series(retrieve_dict)
retrieve_series

b'Student@Chris Evans'           b'\xf8\xbc\xb6>(o\xd4\xbe\x85\xa3 =\xba\x11\x1...
b'Student@Scarlett Johansson'    b'\xc6\x9b\n\xbeut\x90\xbe\xda\xa0\xda\xbe\x17...
b'Ahikmah@'                      b'\xcf\xb5\xfd\xbfv\xecu=\xa5\xd16\xbeL\x1e:=`...
b'Teacher@Barack Obama'          b'\xd8\x19c\xbez\xc1\xc9?\xba\xc9\x00\xbf\xf2F...
b'Teacher@Morgan Freeman'        b'*FC?X\xffc?\x1a\xfe\xef>)\x99\x84>\xee\xfcb\...
b'Student@Angelina Jolie'        b'K\x97K>\xfd\xc8\xbc?\xa0\x9e\xc8\xbd\xcdha\x...
dtype: object

In [11]:
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x: x.decode('utf-8'), index))

In [12]:
retrieve_series.index = index

In [13]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['role_name', 'facial_features']
retrieve_df

,role_name,facial_features
0,Student@Chris Evans,"[0.35691047, -0.41491055, 0.039218444, -0.1543..."
1,Student@Scarlett Johansson,"[-0.13535985, -0.2821385, -0.42700845, -0.2825..."
2,Ahikmah@,"[-1.9821109, 0.06003996, -0.1785341, 0.0454390..."
3,Teacher@Barack Obama,"[-0.22177827, 1.5762169, -0.5030781, 0.1360128..."
4,Teacher@Morgan Freeman,"[0.76278937, 0.890615, 0.46873552, 0.25898102,..."
5,Student@Angelina Jolie,"[0.19881932, 1.4748837, -0.0979588, -0.8805054..."


## Registration Form

In [14]:
import cv2
from insightface.app import FaceAnalysis

In [15]:
# configure face analysis
app = FaceAnalysis(name='buffalo_l', root='', providers=['CUDAExecutionProvider'])

app.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


#### Collect person information

In [16]:
# python input
name = input('Enter your name: ')
role = input("""
Please select your role:
1. Student
2. Teacher
             
Enter your role:
 """)


if role in ('1', '2'):
    if role == '1':
        role = 'Student'
    else:
        role = 'Teacher'
else:
    print('Invalid role. Please try again.')
        
key = role+'@'+name
print('Your name is: ', name)
print('Your role is: ', role)
print('Your key is: ', key)


Your name is:  Awaliyatul Hikmah
Your role is:  Student
Your key is:  Student@Awaliyatul Hikmah


#### Collect the sample using video capture

In [17]:
cap = cv2.VideoCapture(0) # 0: default camera, 1: external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    
    if not ret:
        print('Error: Unable to capture video')
        break

    # get result from face analysis
    results = app.get(frame, max_num=1) # max_num: maximum number of faces to detect
    
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) # 2: thickness

        # get facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)

    if sample >= 200:
        break

    # show the frame
    cv2.imshow('frame', frame)
    
    # exit if q is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'): # 1 ms delay, 0xFF: 8 bit binary number
        break
cap.release()
cv2.destroyAllWindows()

d:\aloka-vision\venv\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [18]:
len(face_embeddings)

200

#### Optimizing the sample

In [19]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [20]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [21]:
# save key and facial bytes into redis
r.hset(name='academy:register', key=key, value=x_mean_bytes)

1